# Notebook to solve the kaggle competition "Child Mind Institute - Detect Sleep States"
Link to the competition: https://www.kaggle.com/competitions/child-mind-institute-detect-sleep-states


### Install packages

In [1]:
%pip install matplotlib plotly pandas numpy tqdm scikit-learn pyarrow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Imports

In [2]:
from itertools import product
from tqdm import tqdm
from score import score
#from sklearn.svm import SVC
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score, precision_score, average_precision_score
import wandb
from abc import ABC, abstractmethod #, classmethod
from math import ceil, floor
import os
from pathlib import Path
from pickle import dump, load
from custom_enums import ModelTrainingType
import pandas as pd
import pyarrow.parquet as pq
import pyarrow.dataset as ds
import gc

In [3]:

# wandb.login() requires you to get your API key from your account settings
# open the weights and biases website https://wandb.ai/login and login to your account
# then go to your account settings and copy the API key
# paste it in the input box and hit enter

#wandb.login() #TODO: uncomment this line to login to your account

In [4]:
# # start a new wandb run to track this script
# wandb.init(
#     # set the wandb project where this run will be logged
#     project="my-awesome-project",
    
#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": 0.02,
#     "architecture": "CNN",
#     "dataset": "CIFAR-100",
#     "epochs": 10,
#     }
# )

To be able to switch between different models from different libraries at a glance, we implement an interface called `IPipelineRequirements`. This allows us to make the pipleine even more robust and easier to extend upon. 

In [5]:
class IPipelineRequirements(ABC):
    @abstractmethod
    def train(self):
        pass

    @abstractmethod
    def predict(self):
        pass

    @abstractmethod
    def save(self):
        pass

    @abstractmethod
    def load(self):
        pass

    @abstractmethod
    def evaluate(self):
        pass

Our first model is the baseline model, which just takes the mean over all onset and wakeup times and tries to predict `onset` and `wakeup` events with the calculated time.

In [6]:
class BaselineModel(IPipelineRequirements):
    def __init__(self):
        pass

    def train(self):
        raise NotImplementedError("Please Implement this method")

    def predict(self):
        raise NotImplementedError("Please Implement this method")

    def save(self):
        raise NotImplementedError("Please Implement this method")

    def load(self):
        raise NotImplementedError("Please Implement this method")

    def evaluate(self):
        raise NotImplementedError("Please Implement this method")

In [7]:
g = BaselineModel()

Numerous studies have concentrated on applications using `RandomForest`. The primary motivation for this preference is the model's inherent transparency in decision-making processes, which are readily identifiable in such models. Subsequent to the BaselineModel, efforts have been made to abstract models from the Scikit-learn library. Fortunately, the majority of models within their API exhibit consistent implementation patterns, facilitating their integration into the processing pipeline.

In [8]:
class SkLearnModel(IPipelineRequirements):

    def __init__(self, model_path):
        self.load(model_path)

    def __init__(self, model, identifier, scaler=StandardScaler, sk_model_params=None):
        self._model = model
        self._scaler = scaler if not callable(scaler) else scaler() # if scaler is a classpointer, instantiate it
        self.identifier = identifier
        self._model_params = sk_model_params

    def train(self, X, y, not_scaled=False, **kwargs):
        if not_scaled:
            X = self._scaler.transform(X)

        # due to the big dataset we need to check wich model was instantiated and do some model
        # specific stuff to enable us to train the model in batches
        if isinstance(self._model, RandomForestClassifier):
            new_estimators = kwargs['add_estimators'] if 'add_estimators' in kwargs else 50
            if self._model.warm_start: self._model.n_estimators += new_estimators
            #print(f'Current estimator increased to {self._model.n_estimators}, {new_estimators} added this round.')
        # if isinstance(self._model, SVC):
        #     pass

        self._model.fit(X, y)

    def predict(self, X, not_scaled=False):
        if not self._model: raise ValueError('Please load or train a model first.')
        if not_scaled:
            if not self._scaler: raise ValueError('Please load or fit a scaler first.')
            X = self._scaler.transform(X)
        return self._model.predict(X)

    def save(self):
        try:
            Path.mkdir(Path('models'), exist_ok=False)
            with open(f'models/model_{self.identifier}.pkl', 'wb') as f:
                dump(self._model, f)
            with open(f'models/scaler_{self.identifier}.pkl', 'wb') as f:
                dump(self._scaler, f)
        except:
            raise ValueError('Unable to save model and scaler.')

    def load(self, filepath):
        try:
            # load model and scaler
            if os.path.isfile(filepath):
                print(f'Loading model from {filepath}')
                with open(filepath, 'rb') as f:
                    self._model = load(f)
            scaler_path = f'{os.path.split(filepath)[0]}/scaler_{os.path.split(filepath)[1].split(".")[0].split("_")[1]}.pkl'
            if os.path.isfile(scaler_path):
                print(f'Loading scaler from {scaler_path}')
                with open(filepath, 'rb') as f:
                    self._scaler = load(f)

            # extract identifier from filename
            self._identifier = filepath.split('_')[1].split('.')[0]
        except (FileNotFoundError) as e:
                print(f'File {e} not found')
                raise ValueError('Filepath is not valid. Unable to load model and scaler.')
        except (IndexError) as e:
                print(f'The name of the file does not implement the convention "<model|scaler>_<some identifier>".')
                raise ValueError('Filepath is not valid. Unable to load model and scaler.')
        except (Exception) as e:
            print(f'Something went wrong. {e}')
            raise ValueError('Unknown Error.')

    def evaluate(self, X, y, scoreFx=None, not_scaled=False, prepredicted=None):
        if not scoreFx:
            raise ValueError('Please provide a score function.')
        if not self._model:
            raise ValueError('Please load or train a model first.')
        if not_scaled and prepredicted == None:
            if not self._scaler: raise ValueError('Please load or fit a scaler first.')
            X = self._scaler.transform(X)
            
        #TODO: Does score FX require an array of predictions or a single prediction?
        #TODO: TEST PARAMS OF SCORE FX!!!!
        return scoreFx(y, prepredicted if prepredicted is not None else self._model.predict(X))

### Heuristics method to get the right event under the constraints given by Child Mind Institue

* A single sleep period must be at least 30 minutes in length
* A single sleep period can be interrupted by bouts of activity that do not exceed 30 consecutive minutes
* No sleep windows can be detected unless the watch is deemed to be worn for the duration (elaborated upon, below)
* The longest sleep window during the night is the only one which is recorded
* If no valid sleep window is identifiable, neither an onset nor a wakeup event is recorded for that night.
* Sleep events do not need to straddle the day-line, and therefore there is no hard rule defining how many may occur within a given period. However, no more than one window should be assigned per night. For example, it is valid for an individual to have a sleep window from 01h00–06h00 and 19h00–23h30 in the same calendar day, though assigned to consecutive nights
* There are roughly as many nights recorded for a series as there are 24-hour periods in that series.

In [9]:
def heuristic_filter(data):
    pass

### Import the training and the validation data

In [10]:
raise(ValueError('test'))

ValueError: test

In [11]:
NEW_TRAIN_DATA = '../data/train/'
NEW_VALIDATION_DATA = '../data/val/'
TRAIN_DATA = '../data/train_20231021'
#TRAIN_DATA = '../data/train_20231021_20M'
VALIDATION_DATA = '../data/validation_20231021'

# figure out series id mapping
if not 'series_id_mapping' in vars():
    series_id_mapping = {'train': dict(), 'validation': dict()}
    t = ds.dataset(NEW_TRAIN_DATA).to_table(columns=['series_id'])
    #t = pq.ParquetDataset(TRAIN_DATA).read(columns=['series_id'])
    for i, data in enumerate(t.to_pandas()['series_id'].unique()):
        series_id_mapping['train'][data] = i
    #v = pq.ParquetDataset(VALIDATION_DATA).read(columns=['series_id'])
    v = ds.dataset(NEW_VALIDATION_DATA).to_table(columns=['series_id'])
    for i, data in enumerate(v.to_pandas()['series_id'].unique()):
        series_id_mapping['validation'][data] = i
    del t, v
    gc.collect()

#train_dataset_length = pq.ParquetFile(TRAIN_DATA).metadata.num_rows
train_dataset_length = ds.dataset(NEW_TRAIN_DATA).count_rows()
validation_dataset_length = ds.dataset(NEW_VALIDATION_DATA).count_rows()

def dataloader_full_dataset(validation=False):
    return pd.read_parquet(VALIDATION_DATA if validation else TRAIN_DATA)

def dataloader(validation=False, batch_size=5_000_000):
    parquet_file = pq.ParquetFile(VALIDATION_DATA if validation else TRAIN_DATA)
    for i in parquet_file.iter_batches(batch_size=batch_size):
        yield i.to_pandas()

def batched_dataloader(validation=False, batch_size=100_000):
    dataset = ds.dataset(NEW_VALIDATION_DATA if validation else NEW_TRAIN_DATA)
    for fragment in dataset.get_fragments():
        batches = fragment.to_batches(batch_size=batch_size) # https://arrow.apache.org/docs/python/generated/pyarrow.dataset.Fragment.html#pyarrow.dataset.Fragment.to_batches
        for batch in batches:
            yield batch.to_pandas()
            del batch
        del fragment


### Run the pipeline

In [12]:
# define all the combinations of models and features
models_and_hyperparams = {
    RandomForestClassifier: {
        'params': {
            'n_estimators': [1000, 1500, 2000],
            'max_depth': [20, 40, 60, None],
            'random_state': [0],
            #'criterion': ['gini', 'entropy', 'log_loss'],
            'warm_start': [True]
        },
        'modeltype': SkLearnModel,
        'scaler': StandardScaler,
        'training_type': ModelTrainingType.BATCH,
        'batch_size': 400_000
    },
    # SVC: {
    #     'params': {
    #         'kernel': ['rbf', 'poly', 'poly', 'poly', 'sigmoid'],
    #         'degree': [3, 4, 5],
    #         'C': [1, 10, 100, 1000],
    #         'gamma': ['scale', 'auto']
    #     },
    #     'modeltype': SkLearnModel,
    #     'scaler': StandardScaler
    # },
    # KNeighborsClassifier: {
    #     'params': {
    #         'n_neighbors': [5, 10, 15, 20],
    #         'weights': ['uniform', 'distance'],
    #         'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    #         'leaf_size': [10, 20, 30, 40, 50],
    #         'p': [1, 2]
    #     },
    #     'modeltype': SkLearnModel,
    #     'scaler': StandardScaler
    # }
}

# define scaler
scaler_type = StandardScaler
pretrained_scalers = dict() # used to store trained scalers for later use

# loop over all combinations and append it to the configurations list
# if there are no hyperparams, just instantiate the model without params
for model_type, hyperparams in models_and_hyperparams.items():
    if len(hyperparams['params']) > 0: # if there are hyperparams, build a dict and pass it to the model as parameters
        fx_param_names, fx_param_values = zip(*hyperparams['params'].items())
        for cartesian_product_values in product(*fx_param_values):
            hyperparam_dict = dict(zip(fx_param_names, cartesian_product_values))

            # create or reuse the scaler specified in the models_and_hyperparams dictionary
            # if hyperparams['scaler'] in pretrained_scalers:
            #     print(f'Using pretrained scaler {hyperparams["scaler"].__name__}')
            #     scaler = pretrained_scalers[hyperparams['scaler']]
            # else:
            #     scaler = hyperparams['scaler']()
            #     print(f'Start fitting scaler {hyperparams["scaler"].__name__}')
            #     for batch in tqdm(dataloader(batch_size=15_000_000)):
            #         X = batch.drop(['wearable_on', 'awake', 'wakeup', 'onset'], axis=1)
            #         X['series_id'] = X['series_id'].map(series_id_mapping['train'])
            #         scaler.partial_fit(X)
            #         del X
            #         gc.collect()
            #     # save the scaler for later use
            #     pretrained_scalers[hyperparams['scaler']] = scaler

            # create the model from the modeltype and the hyperparam_dict
            m = hyperparams['modeltype'](model_type(**hyperparam_dict), \
                                        f'{model_type.__name__}-{"-".join([f"{na}__{str(va)}" for na, va in hyperparam_dict.items()])}'.lower(), \
                                        #pretrained_scalers[hyperparams['scaler']], \
                                        StandardScaler(), \
                                        hyperparam_dict)
            

#TODO: uncemment, standardscaler , not_scaled=True


            
            # init wandb
            # start a new wandb run to track this script
            #cfg = m._model_params.copy()
            #cfg['model'] = m._model.__name__
            #wandb.init(project="classic_models", config=cfg)

            # train model
            rounds = 1
            if hyperparams['training_type'] == ModelTrainingType.BATCH:
                print(f'Model will be traind in batches of {hyperparams["batch_size"]} samples.')
                print(f'Dataset contains {train_dataset_length} samples. {(rounds := train_dataset_length // hyperparams["batch_size"])} batches will be used when fitting the model.')
                if model_type == RandomForestClassifier:
                    print(f'At the end {hyperparam_dict["n_estimators"]} estimators will be fitted. That means, {hyperparam_dict["n_estimators"]//rounds} estimators per batch.')
                    m._model.n_estimators = 0 #set to 0. model will increase the number of estimators in each round
                    # tune hyperparameter 'n_estimators' based on the number of batches
                    attune_estimators_div = hyperparam_dict["n_estimators"] / rounds
                    attuned_estimators = (floor(attune_estimators_div) if attune_estimators_div -0.3 < floor(attune_estimators_div) else ceil(attune_estimators_div)) * rounds
                    print(f'Hyperparameter n_estimators will be adjusted to {attuned_estimators} to fit {attuned_estimators / rounds} estimators per round for each {rounds} batches.')
                    hyperparam_dict["n_estimators"] = attuned_estimators

            print(f'Start training model {m.identifier}')
            for batch in tqdm(batched_dataloader(batch_size=hyperparams['batch_size']) if hyperparams['training_type'] == ModelTrainingType.BATCH else dataloader_full_dataset(), total=rounds+1):
                #X = batch.drop(['wearable_on', 'awake', 'wakeup', 'onset'], axis=1)
                X = batch.drop(['wearable_on', 'awake'], axis=1)
                X['series_id'] = X['series_id'].map(series_id_mapping['train'])
                y = batch['wearable_on']
                args = {}
                if model_type == RandomForestClassifier: args['add_estimators'] = hyperparam_dict['n_estimators']//rounds
                m.train(X, y, not_scaled=False, **args)
                del X
                del y
                gc.collect()
            #y_hat = configured_model.predict(X_test)

            #  do the heuristic part...

            #  evaluate model
            print(f'Start evaluating model {m.identifier}')
            score_value_average_precision_score, score_value_recall, score_value_precision = [], [], []
            for validation in tqdm(batched_dataloader(batch_size=hyperparams['batch_size'], validation=True) if hyperparams['training_type'] == ModelTrainingType.BATCH else dataloader_full_dataset(validation=True), total = validation_dataset_length // hyperparams["batch_size"]):
            #validation = batched_dataloader(validation=True)
                validation['series_id'] = validation['series_id'].map(series_id_mapping['validation'])
                validation_y = validation['wearable_on']
                validation.drop(['wearable_on', 'awake'], axis=1, inplace=True)
                validation_y_hat = m.predict(validation, not_scaled=False)
                # this does the same as scoreFX(y, y_hat). It is wrapped in the models evaluate function.
                # the model itself could also do the prediction if prepredicted is None
                # then, the first argument of the evaluate function would be the X_validation data
                score_value_average_precision_score.append(m.evaluate(None, validation_y, scoreFx=average_precision_score, not_scaled=False, prepredicted=validation_y_hat))
                score_value_recall.append(m.evaluate(None, validation_y, scoreFx=recall_score, not_scaled=False, prepredicted=validation_y_hat))
                score_value_precision.append(m.evaluate(None, validation_y, scoreFx=precision_score, not_scaled=False, prepredicted=validation_y_hat))
            #print(score_value_average_precision_score)
            #print(score_value_recall)
            #print(score_value_precision)
                del validation, validation_y

            print(sum(score_value_average_precision_score) / len(score_value_average_precision_score))
            print(sum(score_value_recall) / len(score_value_recall))
            print(sum(score_value_precision) / len(score_value_precision))
            #  save model
            #m.save()


            # delete model to free up memory
            #del m

            break

            #wandb.finish()
    else:
        print("params in the dictionary cannot be empty. Use the standard values in the dictionary for the model", model_type.__name__)

Model will be traind in batches of 400000 samples.
Dataset contains 102937860 samples. 257 batches will be used when fitting the model.
At the end 1000 estimators will be fitted. That means, 3 estimators per batch.
Hyperparameter n_estimators will be attuned to 1028 to fit 257 batches.
Start training model randomforestclassifier-n_estimators__1000-max_depth__20-random_state__0-warm_start__true


  0%|          | 0/258 [00:00<?, ?it/s]

261it [18:09,  4.17s/it]                         


Start evaluating model randomforestclassifier-n_estimators__1000-max_depth__20-random_state__0-warm_start__true


63it [15:22, 14.64s/it]

0.8276893677792554
0.9999992242027509
0.8276893677782184


In [15]:
m._model.feature_importances_

array([0.06562896, 0.24162419, 0.00584494, 0.00401263, 0.00547788,
       0.00416681, 0.0037552 , 0.12641783, 0.07532501, 0.03562472,
       0.00360454, 0.00924274, 0.00542496, 0.00760753, 0.00858847,
       0.00717628, 0.00656394, 0.00666559, 0.00772053, 0.00575924,
       0.00263182, 0.0033555 , 0.00337475, 0.002605  , 0.00316624,
       0.00862429, 0.00241945, 0.00768832, 0.00253797, 0.00237639,
       0.01113159, 0.00830219, 0.00942224, 0.01227443, 0.01303768,
       0.0062006 , 0.01433257, 0.00814385, 0.00691652, 0.00155602,
       0.00200281, 0.00733743, 0.00794706, 0.01097155, 0.01677126,
       0.00385277, 0.01578598, 0.00360981, 0.00268721, 0.00440259,
       0.00450246, 0.00310375, 0.00243564, 0.00597357, 0.00655171,
       0.00314051, 0.00146662, 0.00634773, 0.00323798, 0.00087716,
       0.00368227, 0.00385025, 0.00163694, 0.00372904, 0.00523477,
       0.00159991, 0.00439451, 0.00133934, 0.00141473, 0.00762776,
       0.00684877, 0.00389545, 0.00842169, 0.00750007, 0.00684

In [17]:
ff = batched_dataloader(validation=True, batch_size=400_000)

In [18]:
fg = next(ff)

In [20]:
fg.drop(['wearable_on', 'awake'], axis=1, inplace=True)

In [23]:
list(zip(fg.columns, m._model.feature_importances_))

[('series_id', 0.06562896311270167),
 ('step', 0.24162418773159003),
 ('anglez', 0.005844941230244825),
 ('enmo', 0.004012630450672348),
 ('hour', 0.0054778798574745815),
 ('minute', 0.00416680561203187),
 ('seconds', 0.0037552039012846206),
 ('day', 0.1264178287853106),
 ('month', 0.0753250145088234),
 ('year', 0.03562471892399695),
 ('anglez_abs', 0.003604538623887983),
 ('anglez_rolling_mean_12', 0.009242742353222762),
 ('anglez_rolling_sum_12', 0.005424962548727194),
 ('anglez_rolling_max_12', 0.007607532756626325),
 ('anglez_rolling_min_12', 0.008588469338068807),
 ('anglez_rolling_std_12', 0.007176275703132694),
 ('anglez_rolling_median_12', 0.006563943245912292),
 ('anglez_rolling_variance_12', 0.006665587363804118),
 ('anglez_rolling_25th_percentile_12', 0.007720528330305993),
 ('anglez_rolling_75th_percentile_12', 0.005759243926694549),
 ('anglez_diff_12', 0.002631820370788326),
 ('anglez_diff_rolling_mean_12', 0.0033555012462752024),
 ('anglez_diff_rolling_sum_12', 0.00337474

In [ ]:
raise ValueError('test')

1. add output of the **BEST** model 1 to train dataset -> [pred_wearable_on] -> must be saved to disk in batches!!!
2. feed the new training to the second model
   1. x of the second model is == ALL but X = batch.drop(['wearable_on', 'awake', 'wakeup', 'onset'], axis=1)
   2. y of the second model is y = batch['awake']

- Do not forget to map the series ID
- All values where pred_wearable_on == 0 must be dropped from the traning set for the second model...
  this can be done by adding a filter to the parquet reader like so:
  ```python
  table = pq.read_table("example.parquet",
                      columns=["col1"],
                      filters=[
                          ("col1", ">", 5),
                          ("col1", "<", 10),
                      ])
  ```


1. the output of the **BEST** second model [pred_awake] must be passed through the heuristics fx alongside with.
   1. the following datafields ['series_id', 'ALL TIME FIELDS', 'pred_awake, ??? others???] 


## Duplication of first pipeline

This pipelines purpos is to train the second model

In [ ]:
# define all the combinations of models and features
models_and_hyperparams = {
    RandomForestClassifier: {
        'params': {
            'n_estimators': [200, 300, 400],
            'max_depth': [10, 20, 30, None],
            'random_state': [0],
            'criterion': ['gini', 'entropy', 'log_loss'],
            'warm_start': [True]
        },
        'modeltype': SkLearnModel,
        'scaler': StandardScaler,
        'training_type': ModelTrainingType.BATCH,
        'batch_size': 5_000_000
    }
}

# define scaler
scaler_type = StandardScaler
pretrained_scalers = dict() # used to store trained scalers for later use

# loop over all combinations and append it to the configurations list
# if there are no hyperparams, just instantiate the model without params
for model_type, hyperparams in models_and_hyperparams.items():
    if len(hyperparams['params']) > 0: # if there are hyperparams, build a dict and pass it to the model as parameters
        fx_param_names, fx_param_values = zip(*hyperparams['params'].items())
        for cartesian_product_values in product(*fx_param_values):
            hyperparam_dict = dict(zip(fx_param_names, cartesian_product_values))

            # create or reuse the scaler specified in the models_and_hyperparams dictionary
            # if hyperparams['scaler'] in pretrained_scalers:
            #     print(f'Using pretrained scaler {hyperparams["scaler"].__name__}')
            #     scaler = pretrained_scalers[hyperparams['scaler']]
            # else:
            #     scaler = hyperparams['scaler']()
            #     print(f'Start fitting scaler {hyperparams["scaler"].__name__}')
            #     for batch in tqdm(dataloader(batch_size=15_000_000)):
            #         X = batch.drop(['wearable_on', 'awake', 'wakeup', 'onset'], axis=1)
            #         X['series_id'] = X['series_id'].map(series_id_mapping['train'])
            #         scaler.partial_fit(X)
            #         del X
            #         gc.collect()
            #     # save the scaler for later use
            #     pretrained_scalers[hyperparams['scaler']] = scaler

            # create the model from the modeltype and the hyperparam_dict
            m = hyperparams['modeltype'](model_type(**hyperparam_dict), \
                                        f'{model_type.__name__}-{"-".join([f"{na}__{str(va)}" for na, va in hyperparam_dict.items()])}'.lower(), \
                                        #pretrained_scalers[hyperparams['scaler']], \
                                        StandardScaler(), \
                                        hyperparam_dict)
            

#TODO: uncemment, standardscaler , not_scaled=True


            
            # init wandb
            # start a new wandb run to track this script
            #cfg = m._model_params.copy()
            #cfg['model'] = m._model.__name__
            #wandb.init(project="classic_models", config=cfg)

            # train model
            if hyperparams['training_type'] == ModelTrainingType.BATCH:
                print(f'Model will be traind in batches of {hyperparams["batch_size"]} samples.')
                print(f'Dataset contains {train_dataset_length} samples. {(rounds := train_dataset_length // hyperparams["batch_size"])} batches will be used when fitting the model.')
                if model_type == RandomForestClassifier:
                    print(f'At the end {hyperparam_dict["n_estimators"]} estimators will be fitted. That means, {hyperparam_dict["n_estimators"]//rounds} estimators per batch.')
                    m._model.n_estimators = 0 #set to 0. model will increase the number of estimators in each round

            print(f'Start training model {m.identifier}')
            for batch in tqdm(dataloader(batch_size=hyperparams['batch_size']) if hyperparams['training_type'] == ModelTrainingType.BATCH else [dataloader_full_dataset()]):
                X = batch.drop(['wearable_on', 'awake', 'wakeup', 'onset'], axis=1)
                X['series_id'] = X['series_id'].map(series_id_mapping['train'])
                y = batch['awake']
                args = {}
                if model_type == RandomForestClassifier: args['add_estimators'] = hyperparam_dict['n_estimators']//rounds
                m.train(X, y, not_scaled=False, **args)
                del X
                del y
                gc.collect()

            #y_hat = configured_model.predict(X_test)

            #  do the heuristic part...

            #  evaluate model
            print(f'Start evaluating model {m.identifier}')
            validation = dataloader_full_dataset(validation=True)
            validation['series_id'] = validation['series_id'].map(series_id_mapping['validation'])
            validation_y = validation['awake']
            validation.drop(['wearable_on', 'awake', 'wakeup', 'onset'], axis=1, inplace=True)
            validation_y_hat = m.predict(validation, not_scaled=False)
            # this does the same as scoreFX(y, y_hat). It is wrapped in the models evaluate function.
            # the model itself could also do the prediction if prepredicted is None
            # then, the first argument of the evaluate function would be the X_validation data
            score_value_average_precision_score = m.evaluate(None, validation_y, scoreFx=average_precision_score, not_scaled=False, prepredicted=validation_y_hat)
            score_value_recall = m.evaluate(None, validation_y, scoreFx=recall_score, not_scaled=False, prepredicted=validation_y_hat)
            score_value_precision = m.evaluate(None, validation_y, scoreFx=precision_score, not_scaled=False, prepredicted=validation_y_hat)
            print(score_value_average_precision_score)
            print(score_value_recall)
            print(score_value_precision)
            del validation, validation_y

            #  save model
            #m.save()


            # delete model to free up memory
            #del m

            break

            #wandb.finish()
    else:
        print("params in the dictionary cannot be empty. Use the standard values in the dictionary for the model", model_type.__name__)

In [ ]:
#TODO: https://arrow.apache.org/cookbook/py/io.html#id11
# for splitted files 